In [8]:
import numpy as np
import gym
from gym import spaces
import torch
import torch.nn as nn
import torch.optim as optim

ModuleNotFoundError: No module named 'gym'

In [3]:
class LudoEnv(gym.Env):
    def __init__(self):
        super(LudoEnv, self).__init__()
        self.num_pieces = 4
        self.track_length = 57
        self.action_space = spaces.Discrete(self.num_pieces)
        self.observation_space = spaces.Box(low=0, high=self.track_length, shape=(8,), dtype=np.int32)
        self.reset()

    def reset(self):
        self.positions = {
            1: np.zeros(self.num_pieces, dtype=np.int32),
            2: np.zeros(self.num_pieces, dtype=np.int32)
        }
        self.done = False
        self.current_player = 1
        return self._get_obs()

    def _get_obs(self):
        current = self.positions[self.current_player]
        opponent = self.positions[3 - self.current_player]
        return np.concatenate([current, opponent])
    
    def step(self, action):
        if self.done:
            return self._get_obs(), 0, True, False, {}

        dice = np.random.randint(1, 7)
        reward = 0
        player = self.current_player

        if self.positions[player][action] < self.track_length:
            self.positions[player][action] += dice
            if self.positions[player][action] >= self.track_length:
                self.positions[player][action] = self.track_length
                reward = 50
            else:
                reward = 1

        if np.all(self.positions[player] >= self.track_length):
            self.done = True
            reward = 100

        self.current_player = 3 - self.current_player
        return self._get_obs(), reward, self.done, False, {}


NameError: name 'gym' is not defined

In [4]:
class DQN(nn.Module):
    # Initializes the neural network layers.
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 256) # First fully connected layer that maps state to 256 hidden units
        self.fc2 = nn.Linear(256, 256) # Second fully connected layer that processes the 256 hidden units
        self.fc3 = nn.Linear(256, action_size) # Output layer that maps to the Q-values of all possible actions

    #  Defines the forward pass of the network.
    def forward(self, state):
        x = torch.relu(self.fc1(state)) # Apply ReLU activation to the output of the first layer
        x = torch.relu(self.fc2(x)) # Apply ReLU activation to the output of the second layer
        return self.fc3(x) # Output Q-values from the final layer 

In [5]:
class ReplayBuffer:
    # Initialize the buffer with a given capacity
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)# Using deque to store experiences, with a maximum length (capacity)
     # Add a new experience to the buffer (state, action, reward, next_state, done)
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done)) # Append the experience to the buffer

    # Sample a batch of experiences from the buffer
    def sample(self, batch_size):
        transitions = random.sample(self.buffer, batch_size) # Randomly select a batch of experiences
        return zip(*transitions) # Return the experiences separated into different components 

    # Return the current size of the buffer
    def __len__(self):
        return len(self.buffer)


In [ ]:
import matplotlib.pyplot as plt

from gym.envs.registration import register
register(id='Ludo-v0', entry_point=LudoEnv)


# make a copy of environment
env = gym.make('Ludo-v0')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# add data of players
players = {
    1: {
        #policy network to dicision making
        "policy": DQN(8, 4).to(device),

        # use to update values 
        "target": DQN(8, 4).to(device),

        # use to update waits
        "optimizer": None,
        #Replay Buffer لتخزين تجارب اللاعب (state, action, reward, ...)
        "buffer": ReplayBuffer(10000),
        #list to save total reword
        "rewards": [],
        # هل اللاعب وصل لحالة استقرار؟ (True/False)
        "converged": False,
        "convergence_ep": -1
    },
    2: {
        "policy": DQN(8, 4).to(device),
        "target": DQN(8, 4).to(device),
        "optimizer": None,
        "buffer": ReplayBuffer(10000),
        "rewards": [],
        "converged": False,
        "convergence_ep": -1
    }
}

for p in players:

    #copy of policy to target
    players[p]["target"].load_state_dict(players[p]["policy"].state_dict())
    players[p]["target"].eval()

    #optimizer to update policy
    players[p]["optimizer"] = optim.Adam(players[p]["policy"].parameters(), lr=0.001)
    gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay = 500
target_update_freq = 10
batch_size = 64
episodes = 500

for episode in range(episodes):
    state = env.reset()
    state = torch.FloatTensor(state).to(device)
    done = False
    rewards = {1: 0, 2: 0}
    current_player = 1

    while not done:
        epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-1. * episode / epsilon_decay)
        if random.random() < epsilon:
            action = random.randint(0, 3)
        else:
            with torch.no_grad():
                q_values = players[current_player]["policy"](state)
                action = q_values.argmax().item()

        next_state, reward, done, _ = env.step(action)
        next_state_tensor = torch.FloatTensor(next_state).to(device)

        players[current_player]["buffer"].push(state.cpu().numpy(), action, reward, next_state, done)
        rewards[current_player] += reward
        state = next_state_tensor
        current_player = 3 - current_player

